In [1]:
# 라이브러리 임포트
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import json
import csv
import os
from datetime import datetime

print("="*80)
print("Base Model (Qwen2.5-1.5B-Instruct) 테스트 시작")
print("="*80)

Base Model (Qwen2.5-1.5B-Instruct) 테스트 시작


In [2]:
# 설정
MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"  # Base 모델
OUTPUT_DIR = "./test_results"  # 결과 저장 경로
CSV_FILE = "htp_interpretations_combined.csv"  # CSV 파일 경로

os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"모델: {MODEL_NAME}")
print(f"결과 저장: {OUTPUT_DIR}")
print(f"CSV 파일: {CSV_FILE}")

모델: Qwen/Qwen2.5-1.5B-Instruct
결과 저장: ./test_results
CSV 파일: htp_interpretations_combined.csv


## 1. Base 모델 로드

In [3]:
print("\n[1/4] Base 모델 로드 중...")

try:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
    )
    
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_NAME,
        trust_remote_code=True,
        padding_side="right",
    )
    
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    print(f"✅ 모델 로드 완료: {MODEL_NAME}")
    print(f"   Device: {model.device}")
    print(f"   VRAM: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
    
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    raise


[1/4] Base 모델 로드 중...


`torch_dtype` is deprecated! Use `dtype` instead!


✅ 모델 로드 완료: Qwen/Qwen2.5-1.5B-Instruct
   Device: cuda:0
   VRAM: 3.09 GB


## 2. 추론 함수 정의

In [4]:
def generate_htp_interpretation(instruction, image_caption, max_tokens=256, verbose=False):
    """
    HTP 해석 생성 함수
    
    Args:
        instruction: 해석 요청 지시문
        image_caption: 이미지 캡션 (HTP 그림 설명)
        max_tokens: 최대 생성 토큰 수
        verbose: 상세 출력 여부
    
    Returns:
        생성된 심리학적 해석 텍스트
    """
    messages = [
        {
            "role": "system",
            "content": "You are an expert psychologist specialized in HTP (House-Tree-Person) test interpretation. "
                      "Provide detailed and insightful psychological analysis based on the given image caption."
        },
        {
            "role": "user",
            "content": f"{instruction}\n\nImage caption: {image_caption}"
        }
    ]
    
    # 프롬프트 생성
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    if verbose:
        print(f"\n[Prompt (처음 200자)]:\n{prompt[:200]}...\n")
    
    # 토크나이즈
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # 생성
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.8,
            top_p=0.95,
            top_k=40,
            do_sample=True,
            repetition_penalty=1.2,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    # 디코딩 (새로 생성된 부분만)
    input_length = inputs['input_ids'].shape[1]
    generated_tokens = outputs[0][input_length:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
    
    return response.strip()

print("✅ 추론 함수 정의 완료")

✅ 추론 함수 정의 완료


## 3. 테스트 케이스 실행

In [5]:
print("\n[2/4] 테스트 케이스 평가 중...")

test_cases = [
    "The tree is large with many branches and deeply drawn roots",
    "The house is small with no doors or windows, and no smoke from the chimney",
    "The person's hands are hidden behind their back and the face has no expression",
    "The tree trunk has holes and the branches are drooping downward",
    "The house has an excessively large roof with many complex decorations",
    "The tree is drawn in the corner with very thin trunk and sparse leaves",
    "The person is drawn very small at the bottom of the page with no facial features",
    "The house has heavily shaded walls and windows with bars",
    "The tree has broken branches and appears to be leaning to one side",
    "The person has unusually large hands and feet with exaggerated details"
]

test_results = []


[2/4] 테스트 케이스 평가 중...


In [6]:
# 테스트 실행
for i, test_input in enumerate(test_cases, 1):
    print(f"\n{'='*80}")
    print(f"테스트 케이스 {i}/{len(test_cases)}")
    print(f"입력: {test_input}")
    print('-' * 80)
    
    try:
        interpretation = generate_htp_interpretation(
            instruction="Please provide a psychological interpretation of the following HTP test image caption",
            image_caption=test_input,
            max_tokens=200
        )
        
        print(f"🤖 Base Model 해석:\n{interpretation}")
        
        test_results.append({
            "case_id": i,
            "input": test_input,
            "model_output": interpretation,
            "output_length": len(interpretation),
            "status": "success"
        })
        
    except Exception as e:
        print(f"❌ 생성 오류: {e}")
        test_results.append({
            "case_id": i,
            "input": test_input,
            "status": "failed",
            "error": str(e)
        })

print(f"\n✅ 테스트 완료: {len([r for r in test_results if r['status']=='success'])}/{len(test_cases)} 성공")


테스트 케이스 1/10
입력: The tree is large with many branches and deeply drawn roots
--------------------------------------------------------------------------------
🤖 Base Model 해석:
The HTP Test, also known as the house-tree-person drawing technique, provides insight into one's personality through visual elements such as trees, houses, people, or other objects. In this case, we're analyzing a representation that depicts a massive tree with numerous branches and deep-rooted structure.

**Psychological Interpretation:** 

1. **Strength and Stability**: 
   - A strong presence represented by the 'tree' indicates significant strength and stability in various aspects of life.
   
2. **Roots vs. Branches**:
   - Deeply rooted structures often symbolize grounding oneself firmly in fundamental values, beliefs, and emotional security. This suggests someone who has established solid foundations for their personal philosophy and relationships.

3. **Complexity and Depth**:
    - Many branching levels s

## 4. 결과 저장

In [7]:
print("\n[3/4] JSON 결과 저장 중...")

# JSON 파일로 저장
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
result_file = f"{OUTPUT_DIR}/base_model_test_results_{timestamp}.json"

with open(result_file, 'w', encoding='utf-8') as f:
    json.dump({
        "test_info": {
            "model_name": MODEL_NAME,
            "test_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "total_tests": len(test_results),
            "successful_tests": len([r for r in test_results if r['status']=='success']),
            "model_type": "Base (No Fine-tuning)"
        },
        "test_cases": test_results
    }, f, indent=2, ensure_ascii=False)

print(f"💾 JSON 저장 완료: {result_file}")

# 통계 출력
success_results = [r for r in test_results if r['status']=='success']
if success_results:
    avg_length = sum(r['output_length'] for r in success_results) / len(success_results)
    print(f"\n📊 통계:")
    print(f"   평균 출력 길이: {avg_length:.1f}자")
    print(f"   성공률: {len(success_results)/len(test_cases)*100:.1f}%")


[3/4] JSON 결과 저장 중...
💾 JSON 저장 완료: ./test_results/base_model_test_results_20251117_145654.json

📊 통계:
   평균 출력 길이: 1128.3자
   성공률: 100.0%


## 5. CSV 파일에 추가

In [ ]:
print("\n[4/4] CSV 파일에 결과 추가 중...")

# 기존 CSV 읽기 (여러 인코딩 시도)
existing_data = []
if os.path.exists(CSV_FILE):
    encodings = ['utf-8-sig', 'utf-8', 'cp949', 'euc-kr', 'latin-1']
    for encoding in encodings:
        try:
            with open(CSV_FILE, 'r', encoding=encoding) as f:
                reader = csv.DictReader(f)
                raw_data = list(reader)
                
                # 기존 데이터 형식 변환
                for row in raw_data:
                    # 기존 형식 확인 및 변환
                    if 'model_type' in row:
                        # 이미 새로운 형식
                        existing_data.append(row)
                    else:
                        # 구 형식: 컬럼명이 model_type이 아닌 경우
                        # 첫 번째 컬럼이 model_type으로 추정
                        keys = list(row.keys())
                        if len(keys) >= 3:
                            existing_data.append({
                                'model_type': row[keys[0]],  # 첫 번째 컬럼
                                'input': row[keys[1]],        # 두 번째 컬럼
                                'interpretation': row[keys[2]], # 세 번째 컬럼
                                'ground_truth': row.get(keys[3], '') if len(keys) > 3 else ''
                            })
            
            print(f"   기존 데이터: {len(existing_data)}개 항목 (인코딩: {encoding})")
            break
        except (UnicodeDecodeError, UnicodeError):
            continue
    
    if not existing_data:
        print("   ⚠️ CSV 파일을 읽을 수 없습니다. 새로 생성합니다.")

# 새로운 데이터 추가
new_data_count = 0
for result in success_results:
    # 동일한 input이 있는지 확인
    existing_inputs = [row['input'] for row in existing_data]
    
    if result['input'] not in existing_inputs:
        existing_data.append({
            'model_type': 'BaseModel',
            'input': result['input'],
            'interpretation': result['model_output'],
            'ground_truth': ''  # Base model은 ground truth 없음
        })
        new_data_count += 1
    else:
        print(f"   ⚠️ 이미 존재하는 입력: {result['input'][:50]}...")

# CSV 파일 쓰기
with open(CSV_FILE, 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.DictWriter(f, fieldnames=['model_type', 'input', 'interpretation', 'ground_truth'])
    writer.writeheader()
    writer.writerows(existing_data)

print(f"✅ CSV 업데이트 완료: {CSV_FILE}")
print(f"   추가된 항목: {new_data_count}개")
print(f"   총 항목: {len(existing_data)}개")

print("\n" + "="*80)
print("Base Model 테스트 완료!")
print("="*80)


[4/4] CSV 파일에 결과 추가 중...
   기존 데이터: 10개 항목 (인코딩: cp949)
   ⚠️ 이미 존재하는 입력: The tree is large with many branches and deeply dr...
   ⚠️ 이미 존재하는 입력: The house is small with no doors or windows, and n...
   ⚠️ 이미 존재하는 입력: The person's hands are hidden behind their back an...
   ⚠️ 이미 존재하는 입력: The tree trunk has holes and the branches are droo...
   ⚠️ 이미 존재하는 입력: The house has an excessively large roof with many ...
   ⚠️ 이미 존재하는 입력: The tree is drawn in the corner with very thin tru...
   ⚠️ 이미 존재하는 입력: The person is drawn very small at the bottom of th...
   ⚠️ 이미 존재하는 입력: The house has heavily shaded walls and windows wit...
   ⚠️ 이미 존재하는 입력: The tree has broken branches and appears to be lea...
   ⚠️ 이미 존재하는 입력: The person has unusually large hands and feet with...


ValueError: dict contains fields not in fieldnames: 'LoRa', 'model', 'layer freeze'